# Imaging!

To this point, everything we've done has required some amount of imagination to relate it to the broader astronomy contexts in which it applies. Today we'll go through some exercises and work with real astronomical data to make images that we can process! 

## Warm up
First, let's get into the groove of things by writing a function to read in our data files.

Write a function which takes as its argument a `string` filepath to a `FITS` file, and should have an optional argument to set the extension (default 0). It should then load the given extension of that `fits` file using a [context manager](https://docs.astropy.org/en/stable/io/fits/index.html#opening-a-fits-file), and return a tuple containing the header and data of that extension. 

The function should have documentation that describes the inputs and outputs. Documentation is **incredibly important** when writing code, both in-line and (# comments) and at the top of functions, methods, and classes. 

We'll be using [Sphinx-compatible](https://www.sphinx-doc.org/en/master/) documentation written in the [Numpy/Scipy style](https://numpydoc.readthedocs.io/en/latest/format.html). There are several reasons to do this. 

1. It is a user-friendly and write-friendly, readable documentation format that is easy to add to your functions.
2. It can be rendered by Sphinx, the most popular automatic documentation renderer. If you've ever read the online documentation pages for functions in, e.g., numpy and scipy, those pages were rendered automatically based on the docstrings of the functions in question. This is possible with tools like Sphinx, but the documentation must be formatted correctly for this to work. 

In the cell below, we've provided a dummy function which takes in any number of inputs (mininum 3) and chooses a random input to return. The formatting of the documentation is shown there (as well as in the link above). 

In [ ]:
import numpy as np 

def random_return(a,b,c,*args):
    '''
    A function which requires three inputs which are floats, accepts any number of additional inputs (of any kind), and returns one randomly chosen input. 
    
    Parameters
    ----------
    a: int
        description of this integer input
    b: int
        description of this integer input
    c: int
        description of this integer input
    *args: tuple
        any additional arguments get stored here
    
    Returns
    -------
    choice 
        The randomly selected input (type not specified)
    '''
    full_input_list = [a,b,c] + list(args)
    choice = np.random.choice(full_input_list)
    return choice

In [ ]:
random_return(1,5,4,6,4,21,6)

When our function has been imported in some code, we can use the `help` command to see the documentation at any time:

In [ ]:
help(random_return)

You will also notice the use of `*args` in this function. This allows us to enter additional function arguments. Similar is `**kwargs`, which allows additional arguments tied to an input keyword. The former gets stored in a tuple, while the latter gets stored in a dictionary.

So as a brief overview of documentation, it contains
1. A brief summary of the function 
2. The word Parameters with the next line having underlines of the same length
3. arguments, which are followed by a colon and the data type(s). On the next line, indented, descriptions of the arguments
4. The word Returns, with the same underline scheme
5. The returned objects, labeled the same way as the input. 

Also above, we saw how to format when no data type is specified. There, additional inputs could've been *any* data type, so we can't be sure what the output will be. The main thing we didn't cover is optional arguments. Those are set like this:
```
a: int, optional
    Description of the thing. (default 5)
```
So we mark it as optional, and then give the default for it. 

In [ ]:
from astropy.io import fits 

# your code
def load_fits():
    '''

    '''
    pass

# soln


Using the function you created above, read in the header and data of the file `antenna_Rband.fits`. 

In [ ]:
# Solution


## Plotting

Next, we need to plot the image data. There are several operations that we almost always perform when plotting astronomical data, as well as several user-preferences for how we "by default" plot images before we begin tweaking things. If you spend any time as an astronomer, you will plot quite literally *thousands* of images --- why set all these settings every time, when we can write a handy function to do it for us?

Write a function which takes in as input arguments 
- an image (2D array or masked array) 

as well as the following optional arguments (so set a default)
- figsize (default (15,13) )
- cmap (default 'gray_r')
- scale (default 0.5)
- **kwargs

Inside the function, create figure and axes objects using `plt.subplots()`. When working in notebooks, it is often useful to set the `figsize` argument of subplots to a nice large-ish value, such as `(15,13)`, which will make the image fill most of the notebook. Since *your* function has set figsize as an argument, you can feed `figsize` directly into the `subplots` call, so that a user of the function can leave the default or set their own. 

Next, use `ax.imshow()` to actually plot the image. You'll want to save the output of this, e.g., `im = ax.imshow(...)`. In this plotting call, set `imshow`'s argument `origin='lower'`. We *always* want to do this when dealing with imaging, as we want (0,0) to be a coordinate. 

**By default, matplotlib uses a "matrix-style" plotting, where 0 of the y axis is in the *top* left corner, and 0 of the x axis is in the *bottom* left corner.**

Also within the call to `imshow()`, feed in the cmap from your function (i.e., `cmap=cmap`). The other critical `imshow()` arguments are `vmin` and `vmax`, which sets the saturation points (and thus the contrast) of the image. 

We haven't set `vmin` and `vmax` as arguments of our outer function, but because of `kwargs`, we can still create a default here that can be overriden from outside. 

As a default, within your function, calculate the mean and standard deviation of the image. Set some temporary variables with the quantities `mu - scale*sigma` and `mu + scale*sigma` (where here `mu` is the calculated mean and `sigma` is the calculated std dev, and `scale` was the optional input). Next, check the kwargs dictionary (which will exist in your function because we added the packing argument `**kwargs` to our function. IF `vmin` and `vmax` are in this dictionary, plug those into your imshow command. Otherwise, use the values determined by the calculation above.

Your function should **return** the created `fig` and `ax` objects so the user can continue to tweak them.

Run your function and test its outputs. Once you're satisfied it's working, use it to plot the provided data. Find either a vmin/vmax pair, or a choice of `scale` which makes the image look pretty!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Your code
def implot(...):
    ''' 
    WRITE DOCSTRING HERE
    '''
    pass #replace with your code

In [ ]:
# here's an example of what this might look like for some choice of scale, vmin, and vmax!

So far, we've made it so that with a simple function call and, potentially, with just the input of an image array, we get out a nice plot with a scaling, colormap, and origin selection. In this section, we are going to allow (optionally) for a colorbar to be added to the figure. We're also going to add in the ability for the figure to be plotted in celestial coordinates (i.e., RA and DEC) instead of pixel units, if information about the image (via the world coordinate system, WCS) exists in the image header. 

Add three new optional arguments to your function.
- colorbar = False
- header = None
- wcs = None

Let's start with the colorbar. At the end of your plotting commands, check if `colorbar=True`, and if so, create a colorbar via `plt.colorbar()`, setting the `mappable` argument to whatever you saved the output of `ax.imshow()` into above. Also set the `ax` argument to be your ax; this will tell `matplotlib` to steal a bit of space from that axis to make room for the colorbar. 

In [ ]:
def implot(image,figsize=(15,13),cmap='gray_r',scale=0.5,colorbar=False,header=None,wcs=None,**kwargs):
    # fill this in

In [ ]:
# test your new implot here

Now if we want to plot things in a coordinate system, it's a little more complicated. To do this, we need to first have an `astropy` `WCS` object associated with the image in question. You can import `WCS` from `astropy.wcs`. WCS objects are created from the headers of fits files. In our function, we allow the user to input either a header or a WCS object directly. 

Within your function, check if WCS is input -- if it is, we're good to go and can safely ignore `header` (even if it is provided). If instead only `header` is provided, use the `WCS()` function to create a new wcs object from that header. 

**You'll want to do this at the very top of your function.**


We now need to move our `fig, ax = ....` creation line into an if-statement. If we are using WCS "stuff", you'll need to set a `projection` for your plot that uses the wcs. This is accomplished as follows:

```
fig, ax = plt.subplots(...,subplot_kw={'projection':wcs}) 
```
where `wcs` is whatever you've named the output of `WCS(header)` or is the WCS input directly into the function. 

**Note: In this case, we will get an error from our function that happens because of some distortion coefficient nonsense between astropy and drizzled HST images. If a `header` is passed to your function (i.e., `if header is not None`, you should call this function `strip_SIP()`, which is provided below, to make everything ok -- `header = strip_SIP(header)`).**

In [ ]:
from astropy.wcs import WCS

def strip_SIP(header):
    A_prefixes = [i for i in header.keys() if i.startswith('A_')]
    B_prefixes = [i for i in header.keys() if i.startswith('B_')]
    for a,b in zip(A_prefixes,B_prefixes):
        del header[a]
        del header[b]
    return header

# new implot function below:


If this worked correctly, when you add the header you read in from our image, you should now see the axes of your plot change from pixels to `pos.eq.ra` and `pos.eq.dec`. We're now looking at actual on-sky coordinates!

Within the if-blocks of your function that sets the `ax` to be a wcs projection, set the $x$ and $y$ labels to read "Right Ascension \[hms\]" and "Declination \[degrees\]" in fontsize 15.

Lastly, to polish things off, use `ax.tick_params()` to set inward, larger ticks, and increase the axis tick label size to 15. 

You'll notice (especially with larger ticks) that the are not perpendicular to the axes spines. This is because this particular image has been rotated with respect to the standard celestial coordinate axes. This can be seen more clearly if you add the following to your function:
`ax.coords.grid(color='gray', alpha=0.5, linestyle='solid')`. Try doing that, adding an optional keyword to your function called 'grid' and enabling this command if it is set.

It's taken us some time, but this image could now be placed in a scientific publication. And, since we have a handy function for it, we can make images that look this nice on the fly with a short one line command, yet still have a lot of flexibility over many important inputs. And of course, the figure and axes objects are returned, so one could run this function and then continue tweaking the plot after the fact.

You should also note that once you use the `wcs` projection on some plot, it's no longer a normal ax object, it's a wcsax object. This makes changing certain elements of those axes a little more involved than for a standard one. [This page](https://docs.astropy.org/en/stable/visualization/wcsaxes/ticks_labels_grid.html) and the others linked there are good references. 

## Cutouts
When working with astronomical images, it is often advantageous to be working with a cutout -- a chunk of the image centered on a certain coordinate, and of a certain size. For example, if there is an HII region or star cluster of interest in the above image, we may like to zoom in on it to examine it more closely. 

Now that we've switched over to using celestial coordinates instead of pixels as our projection frame, zooming in on a region is not as simple as slicing our array, e.g., `image[500:700,200:550]`. On the plus side, the framework we'll learn here is very robust, and will allow for all sorts of useful measurements. 

To make a cutout, we'll need the `Cutout2D` module within `astropy`, which we'll import below. To provide the position of the cutout, we need to be able to feed in astronomical coordinates. For this, we'll use `SkyCoord`, a module in `astropy.coordinates`. Finally, we'll need to integrate the `units` module in `astropy` to successfully create coordinate objects.

In [ ]:
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
import astropy.units as u

Let's start with a `SkyCoord` object. There are several coordinate systems used in astronomy, e.g., Galactic coordinates ($b$, $l$), Equatorial ($RA$, $DEC$). The most common (especially in any extragalactic setting) is RA and DEC (as you can see in the image you've plotted already). 

The [documentation](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) for `SkyCoord` is solid, and worth reading. 

The general way we create these objects is, e.g.,

In [ ]:
coord = SkyCoord('12:01:53.6 -18:53:11',unit=(u.hourangle,u.deg))

In this case, the coordinates set above are for NGC 4039, which is the smaller of the two galaxies in the image we're using. 

**If at any point you're trying to make a coordinate object for a well known galaxy/object, try, e.g., `coord = SkyCoord.from_name('NGC 4038')`, and usually that will work!**

In the cell below, use the coordinate we've created, plus a size (use 1x1 arcminutes), and the wcs object for our image, to create a `Cutout2D` object. 

In [ ]:
cutout = # your code here

Now, use your fancy new function to plot the new cutout. 
**Note that Cutout objects contain the image and their own wcs, accessible via, e.g., `cutout.data` and `cutout.wcs`.**

In [ ]:
# your code here

We're now going to do some aperture photometry. 

Aperture Photometry is the process of defining a region on an image (generally, but not always circular), and measuring the sum pixel value within that region. 
The region is known as an aperture, and the "collapsing" of the 2D spatial information about counts in each pixel into a single number is known as photometry.

Below, we've provided a new coordinate, this time centered on the region between the two galaxies. Make a new cutout of that region (again, 1x1 arcmin), and plot it.

In [ ]:
new_coord = SkyCoord('12:01:55.0 -18:52:45',unit=(u.hourangle,u.deg))

In [ ]:
new_cutout = 

In this region, there are a lot of blobby looking roughly circular sources --- Some of the larger ones are HII star forming regions, the smaller ones are likely stars. 

Often, for calibration purposes, we'd need to create apertures around all those sources in the image. We definitely don't want to do that by hand! Instead, we're going to use the `sep` package. 

**Simply `pip install sep` in the next cell to get it installed, you should then be able to import it.**

In [ ]:
import sep

There are three steps to performing aperture photometry with `sep`, which are detailed in [it's documentation](https://sep.readthedocs.io/en/v0.4.x/). 

- Estimate the background 
- Find Sources 
- Perform Aperture Photometry

Using the instructions presented in the documentation linked, measure the background of the cutout image, and run the source extractor on it. Don't forget to subtract the background before running the extractor!

To do this, write a function that takes as input the data (in this case, a `cutout.data` object and a threshold scale (to be multiplied by the `globalrms`, and performs these steps, returning the `objects` array.

When I ran this, I got an error that my "array was not C-contiguous." I found the solution to this issue in [this stackoverflow post](https://stackoverflow.com/a/26782930). Loosely, `sep` uses C-bindings to actually run the heavy lifting code in C rather than Python (it's faster). This means input arrays must be arranged in physical memory the way C is used to. This particular array was not, but it is easy to order it this way. The first line in the function below shows how to do that.

In [ ]:
def run_sep(data,thresh_scale=2.0):
    '''
    Sep wrapper... runs a basic set of sep commands on a given image
    
    Parameters
    ----------
    data: array_like
        the input image
    thresh_scale: float
        a scaling parameter used by sep to determine when to call something a source (see sep documentation)
        
    Returns
    -------
    objects: numpy struct array
        numpy structured array containing the sep-extracted object locations, etc. 
    '''
    data_corder = data.copy(order='C')
    # fill this in
    
    return objects

Run your function and store the output in a variable called `objects`. You should now have an object array containing many detected sources.

In [ ]:
objects = run_sep(new_cutout.data)
len(objects)

The positions of the determined sources are stored in the output structured array and can be indexed with, e.g., `objects['x']` and `objects['y']`. Replot your image of the cutout, but now circle the positions of all your detected sources. Do they line up with actual point sources in the image?

Try using ax.plot, setting your symbol to 'o', your color to 'None', your marker edge color (mec) to some color, and the marker size (ms) to some largeish value, for a quick way to circle objects.

In [ ]:
# plotting code here


It looks like we've done a pretty adequate job finding the sources in this field -- there'a a few that got missed, and a few we might want to remove, but overall, this is pretty solid. There's a lot more that can be done from here, but this was hopefully a good introduction to how we might read in, manipulate, and analyze images for astronomy in python!